## Proof of Concept for Tax-ledger
Taxable transactions Accourding to [Skattverket](https://skatteverket.se/privat/skatter/vardepapper/andratillgangar/kryptovalutor.4.15532c7b1442f256bae11b60.html):
The goal is to make a solid app that can process row excel/csv data from Binance and other exchanges and create pdf for kap 4.D
* **Buying or change against FIAT currency**
* **Swapping (Converting crypto currencies)**
* **If you buy product or service using crypto currency**
*  Borrowing(currently outside this apps scope)
*  Mining(currently outside this apps scope)
*  Staking(Outside Scope)

### How to doanload data from binance 
* **Step 1:** Login
* **Step 2:** Go to **Orders**->**Assets History**
* **Step 3:** Click on download button at top right and the click on **Export Transaction Records**
* **Step 4:** Choose **Excel**(pdf fails as of 2026-02-27) on **Profile format**
* **Step 5:** Choose date range , better from strat until now to calculate correctly

## POC start

In [1]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os

In [2]:
## Functions
FIAT = {"USD", "EUR", "SEK", "GBP"}
def aggregate_by_coin(df):
    if df.empty:
        return pd.DataFrame()
    return (
        df.groupby("Coin", as_index=False)
          .agg({"Change": "sum"})
    )
def split_and_aggregate(gr):
    incoming = gr[gr["Change"] > 0]
    outgoing = gr[gr["Change"] < 0]
    fee = gr[gr["Operation"].str.contains("Fee", case=False, na=False)]

    incoming = aggregate_by_coin(incoming)
    outgoing = aggregate_by_coin(outgoing)
    fee = aggregate_by_coin(fee)

    return incoming, outgoing, fee
def build_transaction(time, incoming, outgoing, fee, operations):
    if len(incoming) != 1 or len(outgoing) != 1:
        return None  # ambiguous → skip safely

    tx = {
        "Time": time,
        "Economic_Type": None,  # inferred below
        "Incoming": incoming["Change"].iloc[0],
        "Incoming_Coin": incoming["Coin"].iloc[0],
        "Outgoing": abs(outgoing["Change"].iloc[0]),
        "Outgoing_Coin": outgoing["Coin"].iloc[0],
        "Fee": abs(fee["Change"].iloc[0]) if not fee.empty else 0,
        "Fee_Coin": fee["Coin"].iloc[0] if not fee.empty else "",
        "Binance_Operations": ",".join(sorted(operations))
    }

    # Infer economic type
    if tx["Outgoing_Coin"] in FIAT and tx["Incoming_Coin"] not in FIAT:
        tx["Economic_Type"] = "ACQUIRE"
    elif tx["Outgoing_Coin"] not in FIAT and tx["Incoming_Coin"] in FIAT:
        tx["Economic_Type"] = "DISPOSE"
    elif tx["Outgoing_Coin"] not in FIAT and tx["Incoming_Coin"] not in FIAT:
        tx["Economic_Type"] = "SWAP"
    else:
        tx["Economic_Type"] = "UNKNOWN"

    return tx

In [3]:
load_dotenv()

file_name = os.getenv("DATA_FILE")
df = pd.read_csv(file_name)

# Drop columns where everything is NaN
df = df.dropna(axis=1, how='all')

# Convert datetime
df['datetime'] = pd.to_datetime(df['Time'], format='%y-%m-%d %H:%M:%S')

# Round to nearest second
df['datetime_rounded'] = df['datetime'].dt.floor('S')

C:\Users\codecrafter\AppData\Local\Temp\ipykernel_5956\2185167455.py:13: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df['datetime_rounded'] = df['datetime'].dt.floor('S')


In [5]:
transactions = []
processed_rows = 0
group_by_dt=df.groupby("datetime_rounded")
for timestamp, gr in group_by_dt:

    # --- split ---
    deposit_rows = gr[gr["Operation"] == "Deposit"]
    economic_rows = gr[gr["Operation"] != "Deposit"]

    # --- emit deposits ---
    for _, d in deposit_rows.iterrows():
        transactions.append({
            "Time": timestamp,
            "Transaction Type": "Deposit",
            "Taxable": False,
            "Outgoing": 0,
            "Outgoing_Coin": "",
            "Incoming": d["Change"],
            "Incoming_Coin": d["Coin"],
            "Fee": 0,
            "Fee_Coin": ""
        })

    processed_rows += len(deposit_rows)

    if economic_rows.empty:
        continue

    operations = set(economic_rows["Operation"])

    # ---- Binance Convert ----
    if operations == {"Binance Convert"}:
        inc, out, fee = split_and_aggregate(economic_rows)
        tx = build_transaction(timestamp, inc, out, fee, operations)
        if tx:
            transactions.append(tx)
            processed_rows += len(economic_rows)
        continue

    # ---- Transaction Related ----
    if operations == {"Transaction Related"}:
        inc, out, fee = split_and_aggregate(economic_rows)
        tx = build_transaction(timestamp, inc, out, fee, operations)
        if tx:
            transactions.append(tx)
            processed_rows += len(economic_rows)
        continue

    # (other cases unchanged)

    # ---- Buy / Sell / Fee ----
    if operations.issubset({"Buy", "Sell", "Fee"}):
        entries = gr
        inc, out, fee = split_and_aggregate(entries)
        tx = build_transaction(timestamp, inc, out, fee, operations)
        if tx:
            transactions.append(tx)
            processed_rows += len(entries)
        continue

    # ---- Transaction Buy / Spend / Fee ----
    if operations.issubset({"Transaction Buy", "Transaction Spend", "Transaction Fee"}):
        entries = gr
        inc, out, fee = split_and_aggregate(entries)
        tx = build_transaction(timestamp, inc, out, fee, operations)
        if tx:
            transactions.append(tx)
            processed_rows += len(entries)
        continue

    # ---- Transaction Revenue / Sold / Fee ----
    if operations.issubset({"Transaction Revenue", "Transaction Sold", "Transaction Fee"}):
        entries = gr
        inc, out, fee = split_and_aggregate(entries)
        tx = build_transaction(timestamp, inc, out, fee, operations)
        if tx:
            transactions.append(tx)
            processed_rows += len(entries)
        continue

In [6]:
new_data=pd.DataFrame(transactions)